In [1]:
from __future__ import print_function
import numpy as np
import random
import sys

In [2]:
text = open('./shakespeare_final.txt', 'rt', encoding='UTF8').read().lower()

In [3]:
characters = sorted(list(set(text)))

In [4]:
print('corpus length:', len(text))
print('total chars:', len(characters))

corpus length: 196788
total chars: 63


In [5]:
char2indices = dict((c, i) for i, c in enumerate(characters))
indices2char = dict((i, c) for i, c in enumerate(characters))

In [6]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 65583


In [7]:
# 인덱스를 벡터화된 형태로 변환
X = np.zeros((len(sentences), maxlen, len(characters)), dtype=np.bool)
y = np.zeros((len(sentences), len(characters)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char2indices[char]] = 1
    y[i, char2indices[next_chars[i]]] = 1

In [8]:
from keras.models import Sequential
from keras.layers import Dense, LSTM,Activation,Dropout
from tensorflow.keras.optimizers import RMSprop

Using TensorFlow backend.


In [9]:
# 모델 구축
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(characters))))

model.add(Dense(len(characters)))

model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.01))

print (model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               98304     
_________________________________________________________________
dense_1 (Dense)              (None, 63)                8127      
_________________________________________________________________
activation_1 (Activation)    (None, 63)                0         
Total params: 106,431
Trainable params: 106,431
Non-trainable params: 0
_________________________________________________________________
None


In [10]:
# 예측을 인덱스로 변환하는 함수
def pred_indices(preds, metric=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / metric
    exp_preds = np.exp(preds)
    preds = exp_preds/np.sum(exp_preds)
    probs = np.random.multinomial(1, preds, 1)
    return np.argmax(probs)

In [11]:
# 모델 학습 및 평가
for iteration in range(1, 30):
    print('-' * 40)
    print('Iteration', iteration)
    model.fit(X, y,batch_size=128,epochs=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.7,1.2]:

        print('\n----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x = np.zeros((1, maxlen, len(characters)))
            for t, char in enumerate(sentence):
                x[0, t, char2indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = pred_indices(preds, diversity)
            pred_char = indices2char[next_index]

            generated += pred_char
            sentence = sentence[1:] + pred_char

            sys.stdout.write(pred_char)
            sys.stdout.flush()
        print("\n하나의 조합 완료\n")

----------------------------------------
Iteration 1
Epoch 1/1
65583/65583 [==============================] - 55s 835us/step - loss: 2.2897

----- diversity: 0.2
----- Generating with seed: "eneral terms of use part
of this license"
eneral terms of use part
of this licenser there there and there sind the pore of there there and there the prome to so the prowith there there there of there there and of there of there there with there so there and there the prome to sert of there it there it there and and there it loud, his there the prower, i his there and there and there there sond there there there there and there me there his dister for there and there there it th
하나의 조합 완료


----- diversity: 0.7
----- Generating with seed: "eneral terms of use part
of this license"
eneral terms of use part
of this licenser malle at at kinds it most your here, on owher.

 [_exincost your not his there so the hele the werk wark of noterpind nod not hat soll?

hamlet. ard itwind frum comper by verous hi

the use of project gutenberg-tm works cannent you more to be doy, di mire is knows of this known my lord, frem thy carther sor voess, i saile rodige. this this blord and lith and mast; the cast will the rights, the cerpe the quatner the posse, forget ang the please hamlet’s harders and the satiers more most at so my grace oredis bus michtribut that will you what is been and where you hearth to in that or his fellowions that whill but in
하나의 조합 완료


----- diversity: 1.2
----- Generating with seed: "the use of project gutenberg-tm works ca"
the use of project gutenberg-tm works cangent, haposlies tet rake him usestien of deterlus?
do e entr. ie’s thou passed, thou but my curits mungly’d with hamaur’s forgorce you not, too the gee it wass, and fellow, sets of volune, atver’d, end _enssting hamlet
dungs, this youctly?

horatio. let mot fruer.

queen. and is , brect roge. so dough and boly?

dangerne.. and i trange.

gueec. [_sing._]
’ty
lo breaqur worls haste sme?

king. my 
하나의 조합 완료

---

65583/65583 [==============================] - 60s 920us/step - loss: 1.3189

----- diversity: 0.2
----- Generating with seed: "her waist, or in the middle of her favou"
her waist, or in the middle of her favourth, and the dead the world and the dead the king of say so the count my day, when the castle the sonerst, and to the country, when the could the project
gutenberg and my lord, you words it as a saights and to provide the king on the castle.

 enter hamlet. then speech, i was not so the beart the world speak to the constants of done, the read the count of the castle the project
gutenberg and where
하나의 조합 완료


----- diversity: 0.7
----- Generating with seed: "her waist, or in the middle of her favou"
her waist, or in the middle of her favour the unwell, and in when the
semsents, no more has makes manys are, to all his fig’d; in a resum’d his take ath with the worls of states in the dear.

hamlet. i would father.

hamlet. my lord, you so seen the grace meres your pardono.

hamlet. 

king. for shall i have sow it the bort all of father’s copy in the fire. i have seen the project gutenbergvel sere it is not, whe honations fell whene you so work of you and
will not hear. where she s
하나의 조합 완료


----- diversity: 1.2
----- Generating with seed: "orway, uncle of young fortinbras, who, i"
orway, uncle of young fortinbras, who, is th’sfowe, i cast and marrows dere he mother, neosthis,—le in ghe eke welce.


 elses. capy.

hamlet. they proamace, lens, sopsice him, hod, both betweeb.

osrec. but, thus, my lord.

hamlet. marky wa. but she ies it do do eyme ubi they say topher isomafe this
phraskion of his.

hamlet. there. lat’d kings comeve arrues your
wite thoigs! he heak there dismamm as as i gt now,
which stoo to arrus st
하나의 조합 완료

----------------------------------------
Iteration 15
Epoch 1/1
65583/65583 [==============================] - 61s 936us/step - loss: 1.2266

----- diversity: 0.2
----- Generating with seed: "will go seek the king. this is the very
"
will go s

works last a pallabre in the part of this works have hearth; and the
matters
of this offence’s part them astaget. that i cannot be soul of his hames of his hame to
sell of the project gutenberg erseth them that is thee for his word his pathed to the foundation of the
project gutenberg to project gutenberg end to
sempen so must with the partle in them and down, to
has and the pat the main 
하나의 조합 완료


----- diversity: 0.7
----- Generating with seed: "ot ever the
soul of nero enter this firm"
ot ever the
soul of nero enter this firm shade went him shall have me, life theem of this incounte
fighous for
i’d his discourse, in thee, as have so
nom warrous all been of this may shall not cape them notemand’s seese where thee? sindly.

hamlet. what we twoll nevery while him sweet heaven you my cause at
well what is thee reparal heartaut be in a more
that i am boty which theeme?

hamlet. so you, and you ame fortaids heaven of hamlet
하나의 조합 완료


----- diversity: 1.2
----- Generating with seed: "o

of our souls? say, why is this? where he he 
somes. in nect. aw yfr bettter might o, now turh repleact and fortinbres., lactries to den, to some very sighthion
doomthyerour love, rove.ut follow us decess. my daggets, witch draqgitian! let me paadion markes or with the shoith, naget the sent they passhifement and eye, with heaven of what at lenders turn; as dhis me; our samemen sure me to me very putes, my
leavene
your
stay. tres mor
하나의 조합 완료

----------------------------------------
Iteration 24
Epoch 1/1
65583/65583 [==============================] - 54s 824us/step - loss: 1.1576

----- diversity: 0.2
----- Generating with seed: "o sleep, perchance to
dream—ay, there’s "
o sleep, perchance to
dream—ay, there’s a provation and confess of this work in the sone ever to the consequence, our this work in the staget to the consequence, our or his particulas, to still, and to the consequence, our this work in the command the coursect of the cirtuen to project gutenberg-tm electis; but we sh

hamlet. what day thee the play. what in the son the manns, at a contach of the cat them and the play, works as the poon and that the play. there as the poon and for
하나의 조합 완료


----- diversity: 0.7
----- Generating with seed: " if it assume my noble father’s person, "
 if it assume my noble father’s person, to stand to the days the bull my dear lumbegnerish scaptant of a fited wath a please contue that this coulle
star weem works look not what should the sonither?

horatio. sir, he perpagery hath hamlet. so hath be them. but been on deemit pours charge not a moneyst coulius, so muchs have never no manners for the refund the ?

follows to hamlet set hands to hears. but have you willipq: and stand a do
하나의 조합 완료


----- diversity: 1.2
----- Generating with seed: " if it assume my noble father’s person, "
 if it assume my noble father’s person, pardtom. anster. thus brood play.

stralong. his ma’d gissom, bight and eonth, affatire, your mancissuces swe the dulain to us i distaken of ?ry l